In [1]:
from nltk import TextTilingTokenizer
from newspaper import fulltext
import requests
from scipy.special import softmax
import torch
from transformers import BertTokenizer, BertForSequenceClassification
import pandas as pd
import numpy as np
from nltk.tokenize import sent_tokenize


## This notebook is meant to familiarize the reader with the procedure of selecting the final model to be used for the purposes of the application
- The training process was repeated four times (available in the 'BERT Fine-tuning' notebook) and four differrent models were exported. The models could only be trained on news headlines and news sentences as there are no labelled databases of news articles' bodies(refer to the 'Model data Generation and Exploration' notebook). Even if such were available, hardware limitations would still prove prohibitive of using entire articles for training the models. Consequently, some measure of their perfromance when classifying entire news articles must be implemented. In that manner, the models' performance coupled with the logic used for evaluating long sequences is tested which is crucial in arguing for the correct funtioning of the application. Considering the aforementioned, the described models' performance is judged not only based on the evaluation metrics extracted in the 'BERT Fine-tuning' notebook but also on the results they generate when scoring the custom dataset (described below). The model that performs best on the custom dataset will be the one used in production. Additionally, the wrongly guessed observations by the best performing model will be scrutinized to explore the weaknesses of the model.
### The notebook is organized as follows:
- The first four cells contain the functions that were built for the application's sentiment extraction functionality. For the ease of the reader they were included in the notebook as well. 
- Custom dataset exploration
- Evaluation of the models
- Final model choice
- Wrong guesses exploration
 - Display of model performance without reducing neutral probabilities

### Functions used for the extraction of sentiment from news articles
- The below four functions were built for the purposes of sentiment extraction and are used in the application

In [11]:
ALLOWED_TOKENS_LENGTH=300
bert_tokenizer=BertTokenizer.from_pretrained('bert-base-cased')

In [12]:
def _get_paragraphs(text):
    '''A function that separates the document(news article) into topical paragraphs. Defensive programming is applied
       to control for two unlikely occurrences. First, in cases where a text is too short TextTilingTokenizer throws
       a ValueError as it cannot detect any topical paragraphs. Second, it could be possible, yet very unlikely, that even
       after a text is split into paragraphs, one or more of them are still longer than the maximum tokens allowed.
       In such cases the paragraph is split into sentences until no paragraph is longer than the maximum tokens allowed
       using _paragraph_to_sent which recursively calls itself until that condition is satisfied.

       Params:
       text: str - the document(news article) to be split into topical paragraphs of length MAX_LENGTH-NUM_SPECIAL_TOKENS

       Returns: list - the document split into topical paragraphs
       '''
    paragraphs=[]
    text_tiling_tokenizer=TextTilingTokenizer()
    try:
        paragraphs=text_tiling_tokenizer.tokenize(text)
    except:
        return _paragraph_to_sent([text])

    return _paragraph_to_sent(paragraphs)

In [13]:
def _paragraph_to_sent(paragraphs):
    """A function that recursively splits paragraphs that are longer than the maximum tokens allowed into sentences

       Params:
       paragraphs: list - list of paragraphs obtained by TextTilingTokenizer

       Returns:
       list - a list of the paragraphs with the paragraphs longer than the maximum tokens allowed split into sentences
       """
    tokenized_paragraphs = [bert_tokenizer.tokenize(paragraph) for paragraph in paragraphs]
    max_length_paragraph = max(len(paragraph) for paragraph in tokenized_paragraphs)

    if max_length_paragraph < ALLOWED_TOKENS_LENGTH:
        return paragraphs
    else:

        '''Split the paragraph exceeding MAX_TOKENS_LENGTH into sentences and then form two new paragraphs from them.
           In very rare cases an article might use certain punctuation(e.g listing many numbers using decimal points
           that will be detected as one sentence by sent_tokenizer and exceed MAX_TOKENS_LENGTH. Such situation will
           trigger infinite recursive calls, hence, when that happens the only detected sentence acquired by sent_tokenizer
           is split in two.'''
        sents = sent_tokenize(max(paragraphs, key=len))
        new_paragraphs = []

        if len(sents)==1:
            new_paragraphs.extend([sents[0][:len(sents[0])//2],sents[0][(len(sents[0])//2):]])
        else:
            mid = len(sents)//2
            new_paragraphs.extend([' '.join(sents[:mid]),' '.join(sents[mid:])])

        paragraphs.remove(max(paragraphs, key=len))
        paragraphs += new_paragraphs
    return _paragraph_to_sent(paragraphs)

In [14]:
def get_sentiment(text,model):
    """"A function that retrieves the sentiment of a document(news article). The class probabilities for each class of each paragraph
        are obtained and averaged to arrive at an overall probability for each class of the entire document(see _get_class_probabilities)

        Params:
        text: str - the text document(news article)

        Returns:
        overall_prediction: str - The class that has the overall highest probability for the article(on the ternary scale of negative/neutral/positive)
        class_probabilities: list -A list of the probabilities for each class obtained by _get_class_probabilities
        """

    classes=['negative','neutral','positive']

    #Split the document in topical paragraphs
    paragraphs = _get_paragraphs(text)

    #Get all the encodings to be fed to the model
    encodings =bert_tokenizer.batch_encode_plus(paragraphs,add_special_tokens=True,padding='max_length',max_length=300,
                                                return_attention_mask=True,return_tensors='pt')

    #Get the model output. torch.no_grad() is used as there is not need to calculcate gradients which is otherwise automatically done by pytorch
    with torch.no_grad():
        output= model(encodings['input_ids'],encodings['attention_mask'])

    #Get the probabilities for each class of the entire document
    class_probabilities = _get_class_probabilities(output)

    #Get the overall prediction for the document based on the index of the maximum probability obtained from np.argmax
    overall_prediction = classes[np.argmax(class_probabilities)]

    return overall_prediction,class_probabilities

In [15]:
def _get_class_probabilities(model_output):
    """A function that get the probabilities for each class of a document by averaging the probabilities for each paragraphs.
       First, the paragraph probabilities are obtained using a softmax function before averaging them to arrive at the final probabilities.
       Observing the model's performance and the semantic structure of news articles, a decision was made to reduce the neutral probabilities
       by 50% to avoid misclassifications (described in detail in the project).

       Params:
       model_output: transformers.modeling_outputs.SequenceClassifierOutput - the output of the BERT model

       Returns:
       class_probabilities: list - a list of len=3 where indexes 0,1,2 correspond to classes negative, neutral, and positive respectively

       """

    #Get the model output logits(output of the fully connected linear layer)
    output_logits = model_output[0]

    #Get the probabilities for each class of each paragraphs
    paragraph_probabilities = softmax(output_logits.numpy(),axis=1)
    # Reduce the neutral probabilities by 50%
    paragraph_probabilities[:, 1] = paragraph_probabilities[:, 1] / 2

    #Get the average probability for each class of the entire document by summing each paragraph probability and dividing by the number of paragraphs
    num_paragraphs = paragraph_probabilities.shape[0]
    class_probabilities = np.sum(paragraph_probabilities,axis=0)/num_paragraphs

    return class_probabilities.tolist()

### Custom Articles Dataset Exploration
- Due to the lack of available labelled datasets of news articles texts, for the purposes of this project a short set for evaluation was created. The set contains news articles, their respective headlines and source and a sentiment classification on  a ternary scale of negative, neutral or positive. The dataset was labelled by the author of this project and four other annotators, all having expertise in the field of finance and economics.

In [16]:
articles =pd.read_csv('../project_datasets/articles_data.csv')
articles.head(10)

,Headline,URL,source,headline_sentiment,sentiment
0,S&P 500 Firms Beef Up Their Cash Piles to Deal...,https://www.bloomberg.com/news/articles/2021-0...,Bloomberg,neutral,neutral
1,"U.S. Stocks Are Mixed, Bonds Steady on FOMC Da...",https://www.bloomberg.com/news/articles/2021-0...,Bloomberg,neutral,positive
2,MicroStrategy triples down on bitcoin,https://www.ft.com/content/c4d23ea3-0838-461b-...,Financial Times,positive,positive
3,"Bitcoin soars above $40,000 after Tudor Jones ...",https://www.cnbc.com/2021/06/14/bitcoin-btc-so...,CNBC,positive,neutral
4,Is Google Stock A Good Long-Term Investment Or...,https://seekingalpha.com/article/4431413-googl...,SeekingAlpha,neutral,negative
5,Stocks Stage Late Rally to Set Another Record ...,https://www.bloomberg.com/news/articles/2021-0...,Bloomberg,positive,positive
6,Bitcoin and other cryptos lose steam as summer...,https://finance.yahoo.com/news/cryptocurrency-...,Yahoo Finance,negative,positive
7,Market Wrap: Attempts to Push Bitcoin Above $4...,https://www.coindesk.com/market-wrap-bitcoin-4...,CoinDesk,negative,negative
8,Is Netflix Stock A Good Buy At $500?,https://www.forbes.com/sites/greatspeculations...,Forbes,neutral,neutral
9,AMC Is The Best-Performing Stock In America: D...,https://www.forbes.com/sites/stephenmcbride1/2...,Forbes,neutral,neutral


Below the reader can observe the dsitribution labels in the dataset. The distribution is balanced between the three classes.

In [8]:
articles.sentiment.value_counts()

negative    28
positive    24
neutral     17
Name: sentiment, dtype: int64

Below the reader can see the number of articles coming from the different sources used

In [9]:
articles.source.value_counts()

CNBC               15
Bloomberg          12
Reuters            12
Financial Times    12
Forbes              8
Yahoo Finance       2
MarketWatch         2
CoinDesk            2
Fool.com            1
Investomania        1
SeekingAlpha        1
Investors.com       1
Name: source, dtype: int64

### Evaluation of the models
- Four of the exported models were evaluated. All four achieved comparative macro averaged f1-scores of around 0.8 during the 10-fold cross-validation training process. They did however show different results on the test portion of the dataset which was reserved for testing the exported models. Below the reader can observe the evaluation metrics generated for each of the four models.

- Subsequently, each of the models is tested on the custom articles dataset and the best perfroming one is selected for use in the application.


===Model: BertModel ===
Test Loss: 0.38

              precision    recall  f1-score   support

    negative       0.90      0.88      0.89       408
     neutral       0.87      0.92      0.89       563
    positive       0.88      0.79      0.83       219

    accuracy                           0.88      1190
    macro avg       0.88      0.86     0.87     1190
    weighted avg    0.88      0.88     0.88     1190

===Model: BertModel2 ===
Test Loss: 0.80

              precision    recall  f1-score   support

    negative       0.90      0.88      0.89       408
     neutral       0.85      0.88      0.87       563
    positive       0.79      0.75      0.77       219

    accuracy                           0.86      1190
    macro avg       0.85      0.84     0.84      1190
    weighted avg    0.86      0.86     0.86      1190

===Model: BertModel3 ===
Test Loss: 1.78

              precision    recall  f1-score   support

    negative       0.73      0.64      0.68       408
     neutral       0.67      0.72      0.69       563
    positive       0.55      0.59      0.57       219

    accuracy                           0.67      1190
    macro avg       0.65      0.65     0.65      1190
    weighted avg    0.67      0.67     0.67      1190

===Model: BertFixedPunct ===
Test Loss: 2.21

              precision    recall  f1-score   support

    negative       0.90      0.85      0.88       408
     neutral       0.83      0.91      0.87       563
    positive       0.85      0.72      0.78       219

    accuracy                           0.86      1190
    macro avg       0.86      0.83     0.84      1190
    weighted avg    0.86      0.86     0.86      1190

In [23]:
# The models are too big so they were stored locally. 
path=('C:/Users/marti/BertModels/')
models=['BertModel','BertModel2','BertModel3','BertFixedPunct']
headers = {'User-Agent': 'Mozilla/5.0'}


In [16]:
# A variable that will store the correct predictions of each model.
all_models_predictions=[]
# Due to the fact that some of the sources in the dataset block web scraping, a variable is used to track how many
# observations were actually evaluated.'''
total_evaluated=0
# A variable for tracking the wrongly predicted articles to be explored later.
wrong_predictions={}
all_wrong_predictions={}
for model_name in models:
    correct_predictions=0
    print('=======================')
    print("===Model:",model_name,"===")
    model_path=path+model_name
    model=BertForSequenceClassification.from_pretrained(model_path)
    for index,url in enumerate(articles.URL):
        try:
            text=fulltext(requests.get(url,headers).text)
            ''' The below condition is implemented to avoid noise from sources that block web scraping. Usually, they respond 
              with a short message of length less than 400. '''
            if len(text)>400:
                total_evaluated+=1
                # Get the overall prediction for the article.
                prediction=get_sentiment(text,model)[0]
                if prediction == articles.sentiment.loc[index]:
                    correct_predictions+=1
                else:
                    wrong_predictions[articles.Headline.iloc[index]]={'correct':articles.sentiment.iloc[index],
                                                                     'predicted':prediction,'url':url}
            
                print("#######")
                print('Headline:',articles.Headline.iloc[index],'\n'
                      'Predicted sentiment:',prediction,'\nLabelled sentiment:',articles.sentiment.iloc[index],'\n'
                      'Article URL: ',url)
                print("#######")
                
                if index ==len(articles.URL)-1:
                    all_models_predictions.append(correct_predictions)
                    all_wrong_predictions[model_name]=wrong_predictions
                    wrong_predictions={}
                    correct_predictions=0
        except:
            pass
                

===Model: BertModel ===
#######
Headline: Bitcoin soars above $40,000 after Tudor Jones endorsement, Musk�s about-face on accepting crypto 
Predicted sentiment: positive 
Labelled sentiment: neutral 
Article URL:  https://www.cnbc.com/2021/06/14/bitcoin-btc-soars-after-musk-says-tesla-could-accept-the-crypto-again.html
#######
#######
Headline: Is Netflix Stock A Good Buy At $500? 
Predicted sentiment: positive 
Labelled sentiment: neutral 
Article URL:  https://www.forbes.com/sites/greatspeculations/2021/05/28/is-netflix-stock-a-good-buy-at-500/?sh=2356f76038cb
#######
#######
Headline: AMC Is The Best-Performing Stock In America: Don�t Buy It 
Predicted sentiment: neutral 
Labelled sentiment: neutral 
Article URL:  https://www.forbes.com/sites/stephenmcbride1/2021/06/15/amc-is-the-best-performing-stock-in-america-dont-buy-it/?sh=11e8309c551f
#######
#######
Headline: Jim Cramer: Be patient with bitcoin, approach the S&P with caution 
Predicted sentiment: neutral 
Labelled sentiment: 

#######
Headline: Apple demolishes earnings expectations, but stock falls after iPhone chip supply warning 
Predicted sentiment: positive 
Labelled sentiment: positive 
Article URL:  https://www.cnbc.com/2021/07/27/apple-aapl-earnings-q3-2021.html
#######
#######
Headline: Panic Is Suddenly Spreading Among Bitcoin, Ethereum, BNB, XRP And Dogecoin Traders Even As The Market Soars Toward A $1.7 Trillion Price 
Predicted sentiment: positive 
Labelled sentiment: positive 
Article URL:  https://www.forbes.com/sites/billybambrough/2021/08/01/panic-is-suddenly-spreading-among-bitcoin-etheruem-bnb-xrp-and-dogecoin-traders-even-as-the-market-soars-toward-a-17-trillion-price/?sh=b51831d755da
#######
#######
Headline: Is Amazon Stock Still a Buy? 
Predicted sentiment: positive 
Labelled sentiment: positive 
Article URL:  https://www.investomania.co.uk/is-amazon-stock-still-a-buy/
#######
#######
Headline: Walt Disney's Streaming Subscriber Count Is Catching Up With Netflix's 
Predicted sentiment:

#######
Headline: World stocks hit record high, powered by Wall Street 
Predicted sentiment: negative 
Labelled sentiment: negative 
Article URL:  https://www.reuters.com/business/global-markets-wrapup-4-graphics-2021-04-09/
#######
#######
Headline: Dow climbs more than 200 points on the first trading day of May, retail stocks lead gains 
Predicted sentiment: positive 
Labelled sentiment: positive 
Article URL:  https://www.cnbc.com/2021/05/02/stock-market-futures-open-to-close-news.html
#######
#######
Headline: Big Tech stocks soar as results still come in from nail-biter election 
Predicted sentiment: negative 
Labelled sentiment: positive 
Article URL:  https://www.cnbc.com/2020/11/04/big-tech-stocks-soar-as-results-still-come-in-from-nail-biter-election.html
#######
#######
Headline: Bitcoin soars to new high above $52,000; sustainability concerns rise 
Predicted sentiment: positive 
Labelled sentiment: positive 
Article URL:  https://www.reuters.com/article/us-crypto-currency-id

#######
Headline: US financial regulator warns against strict cryptocurrency rules 
Predicted sentiment: negative 
Labelled sentiment: negative 
Article URL:  https://www.ft.com/content/ae0d40a1-8a4a-4885-a6a7-b157e27b3311
#######
#######
Headline: The Fed moves up its timeline for rate hikes as inflation rises 
Predicted sentiment: negative 
Labelled sentiment: negative 
Article URL:  https://www.cnbc.com/2021/06/16/fed-holds-rates-steady-but-raises-inflation-expectations-sharply-and-makes-no-mention-of-taper.html
#######
#######
Headline: Stocks are set to continue post-Fed decision losses with Dow futures down 50 points 
Predicted sentiment: negative 
Labelled sentiment: negative 
Article URL:  https://www.cnbc.com/2021/06/16/stock-market-futures-open-to-close-news.html
#######
#######
Headline: Investors should prepare for impact of green stress tests on banks 
Predicted sentiment: positive 
Labelled sentiment: negative 
Article URL:  https://www.ft.com/content/68ebd27a-232e-413a-9

#######
Headline: Walt Disney's Streaming Subscriber Count Is Catching Up With Netflix's 
Predicted sentiment: positive 
Labelled sentiment: negative 
Article URL:  https://www.fool.com/investing/2021/08/13/walt-disneys-streaming-subscriber-count-is-catchin/?source=eptyholnk0000202&utm_source=yahoo-host&utm_medium=feed&utm_campaign=article
#######
#######
Headline: Bitcoin prices have slipped for now, but data suggests a long-term rally is ahead 
Predicted sentiment: negative 
Labelled sentiment: negative 
Article URL:  https://www.cnbc.com/2021/08/27/bitcoin-prices-have-slipped-for-now-but-data-suggests-a-long-term-rally-is-ahead.html
#######
#######
Headline: Ethereum Competitor Cardano Rallies As Its Founder Teases Smart Contracts Launch 
Predicted sentiment: positive 
Labelled sentiment: positive 
Article URL:  https://www.forbes.com/sites/ninabambysheva/2021/08/11/ethereum-competitor-cardano-rallies-as-its-founder-teases-smart-contracts-launch/?sh=1224323eec29
#######
#######
Head

#######
Headline: Wall Street flat with Fed meet in focus 
Predicted sentiment: negative 
Labelled sentiment: negative 
Article URL:  https://www.reuters.com/business/futures-tick-higher-inflation-fears-fade-2021-06-11/
#######
#######
Headline: Bitcoin drops to lowest since Jan; stocks fall before Fed minutes 
Predicted sentiment: negative 
Labelled sentiment: negative 
Article URL:  https://www.reuters.com/business/global-markets-wrapup-4-2021-05-19/
#######
#######
Headline: Ethereum extends gains to rise 8%; bitcoin firms 
Predicted sentiment: positive 
Labelled sentiment: positive 
Article URL:  https://www.reuters.com/technology/ethereum-rises-5-2509-bitcoin-firm-2021-05-31/
#######
#######
Headline: S&P 500 closes at a record, Dow jumps 450 points as stocks rally in the final minutes of trading 
Predicted sentiment: positive 
Labelled sentiment: positive 
Article URL:  https://www.cnbc.com/2021/03/25/stock-market-open-to-close-news.html
#######
#######
Headline: Bitcoin drops as

Below the reader can observe the correct predictions for each of the models tested. Model BertModel performed best scoring 35 out of the 42 evaluated articles for an accuracy of 83%.

In [20]:
NUM_LOOPS=4
total_evaluated = int(total_evaluated/NUM_LOOPS)
for i,model in enumerate(models):
    print('Model {} correctly classified {} ot of {} articles. Its accuracy is: {}'.format(model,all_models_predictions[i],
                                                                                          int(total_evaluated),
                                                                                          all_models_predictions[i]/total_evaluated))

Model BertModel correctly classified 35 ot of 42 articles. Its accuracy is: 0.8333333333333334
Model BertModel2 correctly classified 30 ot of 42 articles. Its accuracy is: 0.7142857142857143
Model BertModel3 correctly classified 31 ot of 42 articles. Its accuracy is: 0.7380952380952381
Model BertFixedPunct correctly classified 29 ot of 42 articles. Its accuracy is: 0.6904761904761905


### Final model selection

- Based on the observed performance the final model chosen to be used in the application is **BertModel**.

### Wrong guesses exploration
Having selected a model, the news articles that were guessed incorrectly by it are evaluated. Below the reader can observe that the model mainly fails when distinguishing between negative/positive and neutral articles. Upon closer observation of the entire texts, what often appears to confuse the model is that two or more perspectives are expressed. 

For example, in article 'After 500% Bitcoin Boom, Data Reveals Why April Could Be Even Bigger For The Bitcoin Price' two completely opposing views are present. The negative one comes second implying that perhaps the model's starts to forget some of the dependencies from earlier parts in the text. Such issues are often presents in sequential models like BERT, especillay ones that use attention mechanisms. 


In [21]:
wrong_predictions = all_wrong_predictions['BertModel']

for k,v in wrong_predictions.items():
    print('======================')
    print(k)
    print('Model predicted: {}. Correct prediction is: {}'.format(v['predicted'],v['correct']))
    print('Source: {}'.format(v['url']))

Bitcoin soars above $40,000 after Tudor Jones endorsement, Musk�s about-face on accepting crypto
Model predicted: positive. Correct prediction is: neutral
Source: https://www.cnbc.com/2021/06/14/bitcoin-btc-soars-after-musk-says-tesla-could-accept-the-crypto-again.html
Is Netflix Stock A Good Buy At $500?
Model predicted: positive. Correct prediction is: neutral
Source: https://www.forbes.com/sites/greatspeculations/2021/05/28/is-netflix-stock-a-good-buy-at-500/?sh=2356f76038cb
Investors should prepare for impact of green stress tests on banks
Model predicted: neutral. Correct prediction is: negative
Source: https://www.ft.com/content/68ebd27a-232e-413a-95c8-3b3dd267daba
After 500% Bitcoin Boom, Data Reveals Why April Could Be Even Bigger For The Bitcoin Price
Model predicted: negative. Correct prediction is: neutral
Source: https://www.forbes.com/sites/billybambrough/2021/03/28/after-500-bitcoin-boom-heres-why-april-could-be-even-bigger-for-the-bitcoin-price/?sh=348f00861f20
Big Tech 

### Display of model performance without reducing neutral probabilities

As described, a decision was made to reduce the neutral class probabilities of evaluated articles by 50%. The argument for that choice is that very often an article would have one or two highly negative or positive paragraphs and then an additional 2 or 3 that are more informative and are evaluated as neutral. Despite the fact that the given article clearly tries to convey a negative/positive connotation, it would be evaluated as neutral.Through reducing the neutral probabilities, articles that indeed contain neutral discourse are not falsely evaluated because their negative and positive probabilities would still remain low and will not have an impact on the final evaluation. On the other hand, in cases where the neutral probability just slightly exceeds the positive or negative one due to the described characteristics of news texts, the given article will not be misclassified as neutral. 

The section below serves to showcase the decreased model performance when the described logic is not applied. The model's accurracy drops by slightly more than 10% which could be considered as a significant decrease.

In [18]:
# Edited function to restore the reduced neutral probability to its original value.
def get_sentiment2(text,model):
    """"A function that retrieves the sentiment of a document(news article). The class probabilities for each class of each paragraph
        are obtained and averaged to arrive at an overall probability for each class of the entire document(see _get_class_probabilities)

        Params:
        text: str - the text document(news article)

        Returns:
        overall_prediction: str - The class that has the overall highest probability for the article(on the ternary scale of negative/neutral/positive)
        class_probabilities: list -A list of the probabilities for each class obtained by _get_class_probabilities
        """

    classes=['negative','neutral','positive']

    #Split the document in topical paragraphs
    paragraphs = _get_paragraphs(text)

    #Get all the encodings to be fed to the model
    encodings =bert_tokenizer.batch_encode_plus(paragraphs,add_special_tokens=True,padding='max_length',max_length=300,
                                                return_attention_mask=True,return_tensors='pt')

    #Get the model output. torch.no_grad() is used as there is not need to calculcate gradients which is otherwise automatically done by pytorch
    with torch.no_grad():
        output= model(encodings['input_ids'],encodings['attention_mask'])

    #Get the probabilities for each class of the entire document
    class_probabilities = _get_class_probabilities(output)
    class_probabilities[1]=class_probabilities[1]*2
    

    #Get the overall prediction for the document based on the index of the maximum probability obtained from np.argmax
    overall_prediction = classes[np.argmax(class_probabilities)]

    return overall_prediction,class_probabilities

In [28]:
path='C:/Users/marti/BertModels/BertModel'
model=BertForSequenceClassification.from_pretrained(path)
wrong_predictions={}
total_evaluated=0
correct_predictions=0
for index,url in enumerate(articles.URL):
    try:
        text=fulltext(requests.get(url,headers).text)
        ''' The below condition is implemented to avoid noise from sources that block web scraping. Usually, they respond 
            with a short message of length less than 400. '''
        if len(text)>400:
            total_evaluated+=1
            # Get the overall prediction for the article.
            prediction=get_sentiment2(text,model)[0]
            if prediction == articles.sentiment.loc[index]:
                correct_predictions+=1
            else:
                wrong_predictions[articles.Headline.iloc[index]]={'correct':articles.sentiment.iloc[index],
                                                                     'predicted':prediction,'url':url}
            
            print("#######")
            print('Headline:',articles.Headline.iloc[index],'\n'
                    'Predicted sentiment:',prediction,'\nLabelled sentiment:',articles.sentiment.iloc[index],'\n'
                    'Article URL: ',url)
            print("#######")
    except Exception as e:
        print(e)
        pass

#######
Headline: Bitcoin soars above $40,000 after Tudor Jones endorsement, Musk�s about-face on accepting crypto 
Predicted sentiment: positive 
Labelled sentiment: neutral 
Article URL:  https://www.cnbc.com/2021/06/14/bitcoin-btc-soars-after-musk-says-tesla-could-accept-the-crypto-again.html
#######
#######
Headline: Is Netflix Stock A Good Buy At $500? 
Predicted sentiment: positive 
Labelled sentiment: neutral 
Article URL:  https://www.forbes.com/sites/greatspeculations/2021/05/28/is-netflix-stock-a-good-buy-at-500/?sh=2356f76038cb
#######
#######
Headline: AMC Is The Best-Performing Stock In America: Don�t Buy It 
Predicted sentiment: neutral 
Labelled sentiment: neutral 
Article URL:  https://www.forbes.com/sites/stephenmcbride1/2021/06/15/amc-is-the-best-performing-stock-in-america-dont-buy-it/?sh=11e8309c551f
#######
#######
Headline: Jim Cramer: Be patient with bitcoin, approach the S&P with caution 
Predicted sentiment: neutral 
Labelled sentiment: neutral 
Article URL:  h

#######
Headline: Apple demolishes earnings expectations, but stock falls after iPhone chip supply warning 
Predicted sentiment: positive 
Labelled sentiment: positive 
Article URL:  https://www.cnbc.com/2021/07/27/apple-aapl-earnings-q3-2021.html
#######
#######
Headline: Panic Is Suddenly Spreading Among Bitcoin, Ethereum, BNB, XRP And Dogecoin Traders Even As The Market Soars Toward A $1.7 Trillion Price 
Predicted sentiment: neutral 
Labelled sentiment: positive 
Article URL:  https://www.forbes.com/sites/billybambrough/2021/08/01/panic-is-suddenly-spreading-among-bitcoin-etheruem-bnb-xrp-and-dogecoin-traders-even-as-the-market-soars-toward-a-17-trillion-price/?sh=b51831d755da
#######
#######
Headline: Is Amazon Stock Still a Buy? 
Predicted sentiment: positive 
Labelled sentiment: positive 
Article URL:  https://www.investomania.co.uk/is-amazon-stock-still-a-buy/
#######
#######
Headline: Forget Netflix (NFLX): 10 Better Streaming Stocks to Buy Now 
Predicted sentiment: positive 


Below the reader can observe the model's accuracy when the neutral probabilities are not reduced.

In [37]:
accuracy = correct_predictions/total_evaluated
accuracy

0.7209302325581395